In [1]:
""" 
Assignmnet: HW2
Course : CSC 635
Program: hw2.ipynb
Author: Rafail Islam
"""
#---------------------------------Imports--------------------------------------
import numpy as np
import pandas as pd
import pprint
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
#------------------------------------------------------------------------------

In [2]:
#---------------------------------Variables------------------------------------
#only dataset for problem 01 
training_data = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False)
] 


In [3]:
#---------------------------------Functions----------------------------

# converting list into dataframe for convenience 
def load_dataset_1(training_data):
    """ this functin load data set for problem 01. it takes trainig_data as argumnet, and returns
    a dataframe as train_df
    
    Parameters
    -----------
    training_data : tuples
        dataset as tuples
    
    Returns
    ----------
    train_df : pandas dataframe
        dataset as dataframe
        
    """
    column= ['level', 'lang', 'tweets', 'phd','class_label']
    train_df = pd.DataFrame(columns=column)
    
    for d in training_data:
        dic = d[0]
        dic['class_label']=d[1]
        df1=pd.DataFrame([dic])
        train_df = train_df.append(df1,ignore_index = True)
        
    return train_df

train_df = load_dataset_1(training_data)

In [4]:
# Find entropy of class
def entropyD(df , label ):
    """ This function calculates endropy of a dataset. 
    
    Parameters
    -----------
    df : dataframe
        dataset as dataframe
        
    label : str
        label of the dataset
        
    Returns
    ----------
    entropy : float
        entropy of the dataset 
    """
    entropy = 0
    # calculate entropy
    for classLabels in df[label].unique(): 
        prob = df[label].value_counts()[classLabels]/len(df)
        entropy += - prob * np.log2(prob)
     
    #print(entropy)
    
    return entropy

# Find entropy of an attribute
def entropyA(df,attribute,label):
    """ This function calculates endropy of an attributes over the class of a dataset. 
    
    Parameters
    -----------
    df : dataframe
        dataset as dataframe
    
    attribute : str
        attribute name 
        
    label : str
        label of the dataset
        
    Returns
    ----------
    entropy : float
        entropy of the given attribute over class of the data set
    """
    # unique values of the given attribute
    attributeValues = df[attribute].unique()
    
    # unique class of the dataset
    classLabels = df[label].unique()
    
    entropyAi = 0
    # entropyA(D) = sum ( |Dj|/|D|* entropy(Dj) )
    for value in attributeValues:
        entropyDj = 0
        # entropy(Dj)
        for cLabel in classLabels:
            # |Dj|
            dj = len(df[attribute][df[attribute]==value][df[label]==cLabel])
            # |D|
            d = len(df[attribute][df[attribute]==value])
            prob = dj/d
            entropyDj += -prob * np.log2(prob+ np.finfo(float).eps) #RuntimeWarning: divide by zero encountered in log
                                                              # to avoid this, I added np.finfo(float).eps
    
        prob = d/len(df)
        entropyAi +=- prob * entropyDj
        
    return np.abs(entropyAi)

# #print(entropyD(df,'class_label'))
# print(entropyA(train_df,'level','class_label'))
# print(entropyA(train_df,'lang','class_label'))
# print(entropyA(train_df,'tweets','class_label'))
# print(entropyA(train_df,'phd','class_label'))



In [5]:
def infoGain(df,attribute_list,label):
    """ This function calculates information gain of all attributes provided.
    
    Parameters
    -----------
    df : dataframe
        dataset as dataframe
    
    attribute_list : list of str
        attribute list 
        
    label : str
        label of the dataset
        
    Returns
    ----------
    name of the columns with maximum information gain amoing the provided attributes
    """
    # calculate entropy
    ent_d = entropyD(df,label)
    ent_hist=[]
    for attr in attribute_list:
        # calculate attributes entropy
        ent_attr = entropyA(df,attr,label)
        ent_hist.append(ent_d - ent_attr )
        #print(ent_hist)
    return df.columns[np.argmax(ent_hist)]

infoGain(train_df,train_df.columns[:-1],'class_label')

'level'

In [6]:
def same_class(df,label):
    """this function detemines the purity of a dataset.
    
    Parameters
    ------------
    df : dataframe
        dataset 
        
    label : str
        label of the dataset
        
    Returns
    ----------
    True/False : bool
    class label : class label
    """
    
    classes, counts = np.unique(df[label],return_counts=True)
    #print(classes,counts)
  
    if(len(classes) <= 1):
        return True,classes[0]
    else:
        return False,classes[np.argmax(counts)]
  
same_class(train_df,'lang')

(False, 'Python')

In [7]:
def get_majority_class(df,label):
    """this function detemines mejority class of a dataset.
    
    Parameters
    ------------
    df : dataframe
        dataset 
        
    label : str
        label of the dataset
        
    Returns
    ----------
    class label : dtype of class label
    """
    
    classes, counts = np.unique(df[label],return_counts=True)
    return classes[np.argmax(counts)]

get_majority_class(train_df,'class_label')

True

In [9]:
# Buid ID3 decision tree
def ID3(df,attribute_list,label): 
    """this function build ID3 decision tree recursively.
    
    Parameters
    ------------
    df : dataframe
        dataset 
        
    attribute_list : list of str
        attribute list
        
    label : str
        label of the dataset
        
    Returns
    ----------
    dt : dictionary of dictionary
        decision tree
    """
    #1 create a node
    dt={}

    # 2. If samples are all of the same class C , 
    # then return N as a leaf node labeled with class C
    purity, class_label = same_class(df,label)
    if purity:
        #print("pure")
        dt = class_label
        return dt

    # 3. if attributes_list is empty, then return n as a leaf node labeled with the majority class
    if  len(attribute_list) == 0:
        #print("attr empty")
        dt = get_majority_class(df,label)
        return dt

    # 4. select test_attribute with most information gain
    test_attribute = infoGain(df,attribute_list,label)
    #print("best\t",test_attribute)

    # 5. Label node N with test_attribute
    dt={test_attribute:None}

    # 6. For each known value ai of test_attribute
    #i. grow a branch from node N for the condition test_attribute = ai
    branch ={}
    branch[None]= get_majority_class(df,label)
    for next_test_attribute in df[test_attribute].unique():
        branch[next_test_attribute] = None 

    #pprint.pprint(branch)

    dt[test_attribute]= branch
  
    for value in df[test_attribute].unique():
        
        #print("value\t",value)
    
        # 6 ii. let S i be the set of samples in D for which test_attribute = ai
        si = df[df[test_attribute] == value].reset_index(drop=True)
        #print(si)
    
        # 6 iii. if S i is empty, then attach a leaf node labeled with the majority class in D
        if len(si)==0 :
            print("empty sub set")
            dt[test_attribute][value]= get_majority_class(df,label)
            return dt
        else:
            dt[test_attribute][value] = ID3(si,si.columns[:-1],si.columns[-1])

    return dt

In [10]:

def classify(sample,dt):
    """this function determine the result of a decision tree
    for a single sample
    
    Parameters
    ------------
    sample : dictionary 
        sample data to predict class 
    
    dt : dictionary of dictionary
        decision tree
    Returns
    ----------
    decision : dtype of leaf node
        label of the predicted class
    """
    
    #Recursively traverse the tree 
    for node in dt.keys():
        
        # for missing data returns majority class
        if node not in sample.keys():
            return dt[node][None]
        
        attribute = sample[node]
        
        # for incurrect data returns majority class
        if attribute not in dt[node].keys():
            return dt[node][None]
        
        dt = dt[node][attribute]
        #pprint.pprint(dt)
        decision = False

        #go until we reach leaf node
        if isinstance(dt,dict) :
            decision = classify(sample, dt)
        else:
            decision = dt
            #print(tree)
            break;                            
        
    return decision

In [11]:
# Here we do all the steps in problem 01
def problem1():
    
    # Load data
    train_df = load_dataset_1(training_data)
    
    # Build decision tree
    dt = ID3(train_df,train_df.columns[:-1],train_df.columns[-1])
    pprint.pprint(dt)
    
    # sample test data
    sample1 = {"level" : "Senior","lang" : "Java","tweets" : "yes",'phd' : 'no'} # true
    sample2 = {"level" : "Junior","lang" : "Java","tweets" : "yes","phd" : "yes"} # false
    sample3 = {"level": "Intern"} # true
    sample4 = {"level": "Senior"} # false
    
    # classify test data
    print(classify(sample1, dt))
    print(classify(sample2, dt))
    print(classify(sample3, dt))
    print(classify(sample4, dt))

problem1()

{'level': {None: True,
           'Junior': {'phd': {None: True, 'no': True, 'yes': False}},
           'Mid': True,
           'Senior': {'tweets': {None: False, 'no': False, 'yes': True}}}}
True
False
True
False


# Problem 02

In [19]:
# Load dataset for problem 02
def load_dataset2():
    """ Load dataset and returns data for training and testing
    Parameters
    --------
    null
    
    Returns
    ----------
    training_data : dataframe
        training dataset
    
    X_test : dataframe
        dataset for testing
    y_test : dataframe
        actual class of testing dataset
    """
    
    #https://www.kaggle.com/uciml/pima-indians-diabetes-database
    dataset  = pd.read_csv('diabetes.csv',delimiter=',')
    dataset.head(5)
    
    y = dataset['Outcome']
    X = dataset.drop('Outcome',axis=1)
    y= y.to_frame()
    
    # split dataset into test and train
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)
    
    # concatenate dataframe to make traing dataset with labels
    train_data = pd.concat([X_train,y_train],axis=1)
    
    return train_data, X_test,y_test

In [20]:
def classification_accuracy(dt,X_test,y_test):
    ''' This function caculate classification accuracy
    
    Parameters
    -----------
    dt : dictionary of dictionaries
        decision tree
    X_test : dataframe
        testing dataset
    
    Y_test : dataframe
        actual class of testing dataset
        
    Returns
    ---------
    Accuracy : float
        Accuracy in percentage
    '''
    # Convert dataframe into dictionaries
    samples = X_test.to_dict('records')
    count = 0
    # make 1D np array
    ytest =np.squeeze(y_test.to_numpy())
    
    for sample, y in zip(samples,ytest):
        # get class for single data (row)
        y_predict = classify(sample,dt)
        
        if y == y_predict:
            count += 1
    return (count/len(X_test))*100

#classification_accuracy(dt,X_test,y_test)

In [21]:
# Here we do all steps for problem 02
def problem2():
    
    # load data
    train_data, X_test, y_test = load_dataset2()
    
    # build decision tree
    dt = ID3(train_data, attribute_list = train_data.columns[:-1], label = train_data.columns[-1])
    
    # print accuracy
    print("Test Accuracy %.2f%%"%(classification_accuracy(dt,X_test,y_test)))
    
problem2()   

#-------------------------------------------------------------------------------------------------------

Test Accuracy 57.79%
